# MLOps in Azure

A notebook version of [README.md](../README.md)

## Prerequisites

* Subscription
* Resource group `learn`
* ML workspace `mlops`
* Compute `cheapest-instance` and `cheapest-cluster`
* VS Code extension [Azure Machine Learning](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.vscode-ai) to set a default ML workspace
* [`azure-cli`](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)

## 1: Create an ML job

In [1]:
!az ml job create --file job.yml

Uploading model (0.0 MBs): 100%|██████████| 4219/4219 [00:00<00:00, 5196.09it/s]


{
  "code": "azureml:/subscriptions/b42b69cf-27c0-4ee2-99a0-a718ebd91945/resourceGroups/learn/providers/Microsoft.MachineLearningServices/workspaces/mlops/codes/0b9b12e1-edda-4b0f-b836-3414ba523e56/versions/1",
  "command": "python train.py --training_data ${{inputs.training_data}} --reg_rate ${{inputs.reg_rate}}",
  "compute": "azureml:cheapest-cluster",
  "creation_context": {
    "created_at": "2023-02-22T13:29:36.291689+00:00",
    "created_by": "Filip Povolný",
    "created_by_type": "User"
  },
  "description": "Train a classification model on diabetes data using a local folder as input.",
  "display_name": "helpful_drawer_flpv2hz63t",
  "environment": "azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
  "environment_variables": {},
  "experiment_name": "diabetes-data-example",
  "id": "azureml:/subscriptions/b42b69cf-27c0-4ee2-99a0-a718ebd91945/resourceGroups/learn/providers/Microsoft.Mac

## 2: Use GH Actions for model training

### 2.1: Create Service Principal

In [ ]:
!az ad sp create-for-rbac \
    --name github-aml-sp \
    --role contributor \
    --scopes /subscriptions/b42b69cf-27c0-4ee2-99a0-a718ebd91945/resourceGroups/learn/providers/Microsoft.MachineLearningServices/workspaces/mlops \
    --sdk-auth

### 2.2: Create GitHub Action Secret

* [mslearn-mlops settings] > Secrets and variables > Actions > New repository secret
* Name: **AZURE_CREDENTIALS**
* Secret: [**\<output from the previous command\>**](#2.1:-create-service-principal)


### 2.3: Create Compute cluster

TODO: create with `azure-cli`, right now check the [README](../README.md#23-create-compute-cluster)


### 2.4: Run GH Action manually

* [mslearn-mlops action](https://github.com/ficinator/mslearn-mlops/actions/workflows/02-manual-trigger-job.yml) > Run workflow > hopefully it runs

## 3: Trigger workflow on PR

In [4]:
%%writefile ../.github/workflows/03-trigger-ml-job-pr.yml
name: Trigger AML job on PR

on: [pull_request]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
    - name: Check out repo
      uses: actions/checkout@main
    - name: Azure login
      uses: azure/login@v1
      with:
        creds: ${{secrets.AZURE_CREDENTIALS}}
    - name: Set up azure-cli
      run: |
        az extension add -n ml -y
        az configure --defaults workspace=mlops group=learn
    - name: Create endpoint
      run: az ml job create --file job.yml

Writing ../.github/workflows/03-pr-trigger-ml-job.yml
